# LSEG Data Libraries API

**Financial Time Series Prediction &mdash; Using Deep Neural Networks**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical intraday data across asset classes via the Eikon Data API,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks` and
* how to apply deep learning techniques based on deep neural networks for time series prediction

## Importing Required Packages

In [2]:
import lseg.data  as ld  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import tensorflow as tf  # Tensorflow
import pickle

The following **Python and package versions** are used.

In [3]:
import sys
print(sys.version)

3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]


In [19]:
ld.__version__

'2.0.0'

In [4]:
np.__version__

'1.26.4'

In [5]:
pd.__version__

'2.2.1'

In [8]:
cf.__version__

'0.17.3'

In [20]:
tf.version.VERSION

'2.10.0'

## Retrieving Intraday Data using LSEG data libraries session

We first define a **small universe of `RICS`** for which to retrieve data.

In [4]:
rics = [
    'SPY',  # S&P 500 ETF
    'AAPL.O',  # Apple stock
    'AMZN.O'  # Amazon stock
]

Second, **intraday data** is retrieved.

In [9]:
try:
    data = pickle.load(open('./lseg_intraday_data.pkl', 'rb'))
except:
    ld.open_session()
    data = ld.get_history(universe=rics, # RICS universe
                        fields="TRDPRC_1", 
                        start='2024-05-02',  # start time
                        end='2024-05-03', # end time
                        interval="1min") # bar length
    ld.close_session()
    pickle.dump(data, open(f'lseg_intraday_data.pkl', 'wb'))

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 960 entries, 2024-05-02 08:00:00 to 2024-05-02 23:59:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY     943 non-null    Float64
 1   AAPL.O  958 non-null    Float64
 2   AMZN.O  953 non-null    Float64
dtypes: Float64(3)
memory usage: 32.8 KB


In [ ]:
data.head()  # first five rows

TRDPRC_1                SPY  AAPL.O  AMZN.O
Timestamp                                  
2024-05-02 08:00:00   503.5  170.67  180.95
2024-05-02 08:01:00  503.51  170.72  181.44
2024-05-02 08:02:00  503.62  170.85  181.56
2024-05-02 08:03:00  503.58   171.0  181.18
2024-05-02 08:04:00  503.61  171.15  181.25

In [25]:
data.tail()  # final five rows

TRDPRC_1                 SPY   AAPL.O  AMZN.O
Timestamp                                    
2024-05-02 23:55:00  506.525  183.665   184.8
2024-05-02 23:56:00   506.51  183.535  184.83
2024-05-02 23:57:00   506.52    183.5  184.84
2024-05-02 23:58:00   506.47   183.45  184.76
2024-05-02 23:59:00   506.49   183.46  184.83

In [11]:
data.dropna(inplace=True)

## Calculating the Log Returns

We next calculate the **log returns** in vectorized fashion.

In [16]:
rets = np.log(data / data.shift(1)).dropna()  # log returns in vectorized fashion

In [28]:
rets.head()

TRDPRC_1                  SPY    AAPL.O    AMZN.O
Timestamp                                        
2024-05-02 08:01:00   0.00002  0.000293  0.002704
2024-05-02 08:02:00  0.000218  0.000761  0.000661
2024-05-02 08:03:00 -0.000079  0.000878 -0.002095
2024-05-02 08:04:00   0.00006  0.000877  0.000386
2024-05-02 08:05:00  0.000099 -0.000175 -0.000662

## Plotting the Data

Using `Cufflinks`, we can plot the normalized financial time series as **line plots** for comparison.

In [12]:
data.normalize().iplot(kind='lines')

c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



## Preparing Lagged Data

The code that follows derives the **lagged data** for every single `RIC`. First, a function that adds columns with lagged data to a `DataFrame` object.

In [13]:
lags = 10

In [14]:
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(rets[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        # creates the lagged data column with directional values
        df[col] = df[ric].shift(lag)
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

Second, the iterations over all `RICs`, using the `add_lags` function and storing the resulting `DataFrame` objects in a dictionary.

In [17]:
dfs = {}
for ric in rics:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [18]:
cols  # the column names for the lags

['lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10']

In [19]:
dfs.keys()  # the keys of the dictonary

dict_keys(['SPY', 'AAPL.O', 'AMZN.O'])

In [90]:
dfs['AAPL.O'].head(7)

AAPL.O     lag_1     lag_2     lag_3     lag_4  \
Timestamp                                                               
2024-05-02 08:11:00  0.000234 -0.000935 -0.000759  0.000175  0.000117   
2024-05-02 08:12:00  0.000409  0.000234 -0.000935 -0.000759  0.000175   
2024-05-02 08:13:00  0.000234  0.000409  0.000234 -0.000935 -0.000759   
2024-05-02 08:14:00       0.0  0.000234  0.000409  0.000234 -0.000935   
2024-05-02 08:15:00       0.0       0.0  0.000234  0.000409  0.000234   
2024-05-02 08:16:00 -0.000175       0.0       0.0  0.000234  0.000409   
2024-05-02 08:17:00  0.000351 -0.000175       0.0       0.0  0.000234   

                        lag_5     lag_6     lag_7     lag_8     lag_9  \
Timestamp                                                               
2024-05-02 08:11:00  0.000818 -0.000175  0.000877  0.000878  0.000761   
2024-05-02 08:12:00  0.000117  0.000818 -0.000175  0.000877  0.000878   
2024-05-02 08:13:00  0.000175  0.000117  0.000818 -0.000175  0.000877   
2024-05-02 08:14:00 -0.000759  0.000175  0.000117  0.000818 -0.000175   
2024-05-02 08:15:00 -0.000935 -0.000759  0.000175  0.000117  0.000818   
2024-05-02 08:16:00  0.000234 -0.000935 -0.000759  0.000175  0.000117   
2024-05-02 08:17:00  0.000409  0.000234 -0.000935 -0.000759  0.000175   

                       lag_10  
Timestamp                      
2024-05-02 08:11:00  0.000293  
2024-05-02 08:12:00  0.000761  
2024-05-02 08:13:00  0.000878  
2024-05-02 08:14:00  0.000877  
2024-05-02 08:15:00 -0.000175  
2024-05-02 08:16:00  0.000818  
2024-05-02 08:17:00  0.000117

In [91]:
np.digitize(dfs['AAPL.O'].head(7), bins=[0])

array([[1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]], dtype=int64)

In [22]:
2 ** lags  # number of patterns

1024

## The DNN Model

The matrix consisting of the lagged data columns is used to "predict" the next day's direction of movement of the `RIC` via a **deep neural network (DNN)** algorithm. This is a **classification algorithm** that is able to **learn from historical patterns** (10 lags) to predict whether an upwards movement is more likely or a downwards movement.

In what follows, the `Tensorflow` package from Google is used (see https://www.tensorflow.org/).

<img src="http://hilpisch.com/images/tensorflow_logo.png" width="15%" align="left">

In [23]:
tf.get_logger().setLevel('ERROR')

## The Model Fitting

In [135]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

All DNNs can be trained ("fitted") to the data. The the **DNN model object** is instantiated with three hidden layers. We also save a dictionary of models for the entire process. We use **train_test_split** to generate data for both in sample and out of sample testing.

In [138]:
%%time
models_dict = {}
for ric in rics:
    df = dfs[ric] 
    model = tf.keras.Sequential([tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(loss=tf.keras.losses.binary_crossentropy, 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
              metrics = [tf.keras.metrics.binary_accuracy])

    X = np.digitize(df[df.columns[1:]].values, bins=[0])
    y = np.digitize(df[df.columns[0]].values, bins=[0])
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model.fit(X_train, y_train, epochs=250)
    
    models_dict[ric] = {'model':model,
                   'X_train':X_train,
                   'X_test':X_test,
                   'y_train':y_train,
                   'y_test':y_test}

Epoch 1/250
22/22 [==============================] - 1s 2ms/step - loss: 0.7487 - binary_accuracy: 0.4646
Epoch 2/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6920 - binary_accuracy: 0.5310
Epoch 3/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6922 - binary_accuracy: 0.5339
Epoch 4/250
22/22 [==============================] - 0s 3ms/step - loss: 0.6907 - binary_accuracy: 0.5527
Epoch 5/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6893 - binary_accuracy: 0.5209
Epoch 6/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6786 - binary_accuracy: 0.5671
Epoch 7/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6715 - binary_accuracy: 0.5613
Epoch 8/250
22/22 [==============================] - 0s 5ms/step - loss: 0.6717 - binary_accuracy: 0.5758
Epoch 9/250
22/22 [==============================] - 0s 2ms/step - loss: 0.6656 - binary_accuracy: 0.5945
Epoch 10/250
22/22 [==========================

The prediction value is either `+1` for an upwards movement or `-1` for a downwards movement. With regard to a using this as signals for a trading strategy, one **would go long for `+1` and go short for `-1`**.

## Vectorized Backtesting

Let's backtest the performance of the DNN-based trading strategies. Here, vectorization is used for convencience and speed. First, the **strategy returns** which result from multiplying the prediction or position values by the log returns of the respective `RIC`.

In [159]:
for ric in rics:
    in_sample_predictions = models_dict[ric]['model'].predict(models_dict[ric]['X_train'])
    in_sample_prediction_classes = [1 if prob > 0.5 else -1 for prob in np.ravel(in_sample_predictions)]
    dfs[ric]['in_sample'] = in_sample_prediction_classes * dfs[ric][ric][:len(in_sample_prediction_classes)]

22/22 [==============================] - 0s 1ms/step


Second, the visualization of the **cumulative performance**.

In [163]:
for ric in rics:
    dfs[ric][[ric, 'in_sample']].dropna(axis=1).cumsum().apply(np.exp).iplot()

c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



## Out-of-Sample Testing

Next, to get a more realistic picture of the real trading performance to be expected we use the test set of **train test split** to implement **out-of-sample backtesting**.

In [154]:
for ric in rics:
    out_of_sample_predictions = models_dict[ric]['model'].predict(models_dict[ric]['X_train'])
    out_of_sample_prediction_classes = [1 if prob > 0.5 else -1 for prob in np.ravel(out_of_sample_predictions)]
    dfs[ric]['out_of_sample'] = out_of_sample_prediction_classes * dfs[ric][ric][:len(out_of_sample_prediction_classes)]

22/22 [==============================] - 0s 1ms/step


Roughly speaking, the **green part is taken for training**, the **red part for testing**.

In [156]:
for ric in rics:
    dfs[ric][[ric, 'out_of_sample']].dropna(axis=1).cumsum().apply(np.exp).iplot()

c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



c:\Anaconda\envs\pyquants\lib\site-packages\cufflinks\plotlytools.py:117: FutureWarning:

DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.



## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical intraday data (one minute bars)** via the LSEG Data Libraries API,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient,
* **deep learning (DL) techniques** such as **deep neural networks (DNN)** for classification are easily applied by the use of Python and
* that such techniques might be helpful in **predicting the direction of market movements** using a lag- and pattern-based approach.

## LSEG Data API Developer Resources

* [API Pages](https://developers.lseg.com/en/api-catalog) 
* [Q&A Forums](https://community.developers.refinitiv.com/index.html) 

Data Item Browser Application: Type `DIB` into Workspace Search Bar.

* [Article on Chains](https://developers.lseg.com/en/article-catalog/article/simple-chain-objects-ema-part-1)